In [5]:
import tensorflow as tf
from tensorflow import keras
#from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras import regularizers
import gzip
import pickle
import numpy as np

In [6]:
learning_rate = 0.001
epochs = 30
batch_size = 120

In [7]:
f = gzip.open('mnist.pkl.gz', 'rb')
dataset = pickle.load(f,encoding="latin1")

In [8]:
dat=np.array(dataset)
print(dat[1,1].shape)
(x_train, y_train), (x_val, y_val), (x_test, y_test) = dataset

(10000,)


<ipython-input-8-9d04016ce3ca>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dat=np.array(dataset)


In [9]:
x_train.shape

(50000, 784)

In [10]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [11]:
print(y_train[10000])

3


In [12]:
num_classes=10
y_trainc = keras.utils.to_categorical(y_train, num_classes)
y_testc = keras.utils.to_categorical(y_test, num_classes)

In [13]:
y_trainc[10000]

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [14]:
model = Sequential()
model.add(Dense(512, activation='sigmoid', input_shape=(784,)))
model.add(Dense(200,activation='tanh'))
model.add(Dense(50,activation='exponential'))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dense_1 (Dense)             (None, 200)               102600    
                                                                 
 dense_2 (Dense)             (None, 50)                10050     
                                                                 
 dense_3 (Dense)             (None, 10)                510       
                                                                 
Total params: 515080 (1.96 MB)
Trainable params: 515080 (1.96 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
model.compile(loss='categorical_crossentropy',optimizer='RMSprop',metrics=['accuracy'])

In [16]:
history = model.fit(x_train, y_trainc,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_testc)
                    )

Epoch 1/30
417/417 [==============================] - 7s 14ms/step - loss: 0.6955 - accuracy: 0.7832 - val_loss: 0.2794 - val_accuracy: 0.9187
Epoch 2/30
417/417 [==============================] - 6s 15ms/step - loss: 0.2391 - accuracy: 0.9270 - val_loss: 0.2388 - val_accuracy: 0.9255
Epoch 3/30
417/417 [==============================] - 6s 14ms/step - loss: 0.1660 - accuracy: 0.9493 - val_loss: 0.1378 - val_accuracy: 0.9579
Epoch 4/30
417/417 [==============================] - 6s 15ms/step - loss: 0.1269 - accuracy: 0.9621 - val_loss: 0.1213 - val_accuracy: 0.9635
Epoch 5/30
417/417 [==============================] - 6s 14ms/step - loss: 0.1027 - accuracy: 0.9688 - val_loss: 0.1162 - val_accuracy: 0.9656
Epoch 6/30
417/417 [==============================] - 6s 16ms/step - loss: 0.0846 - accuracy: 0.9749 - val_loss: 0.1005 - val_accuracy: 0.9709
Epoch 7/30
417/417 [==============================] - 5s 13ms/step - loss: 0.0726 - accuracy: 0.9774 - val_loss: 0.0921 - val_accuracy: 0.9745

In [17]:
score = model.evaluate(x_test, y_testc, verbose=1) #evaluar la eficiencia del modelo
print(score)
a=model.predict(x_test) #predicción de la red entrenada
print(a.shape)
print(a[1])
print("resultado correcto:")
print(y_testc[1])


313/313 [==============================] - 1s 5ms/step - loss: 0.2877 - accuracy: 0.9799
[0.2877279818058014, 0.9799000024795532]
313/313 [==============================] - 1s 4ms/step
(10000, 10)
[0.0000000e+00 3.4236526e-17 9.9999994e-01 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 5.4601432e-38 0.0000000e+00 0.0000000e+00]
resultado correcto:
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


In [18]:
#Para guardar el modelo en disco
model.save("red.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [19]:
#para cargar la red:
modelo_cargado = tf.keras.models.load_model('red.h5')